In [11]:
from langchain_community.utilities import SQLDatabase
from langchain.chat_models import init_chat_model
from langchain import hub
from langgraph.prebuilt import create_react_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
db = SQLDatabase.from_uri('sqlite:///classicmodels.db')
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['customers', 'employees', 'offices', 'orderdetails', 'orders', 'payments', 'productlines', 'products']


c:\Users\PC\Desktop\sql_agent\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:134: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.SMALLINT'> with reflected arguments ['6']; using no arguments.
  self._metadata.reflect(


In [3]:
print(db.run("SELECT * FROM customers LIMIT 3;"))

[(103, 'Atelier graphique', 'Schmitt', 'Carine ', '40.32.2555', '54, rue Royale', None, 'Nantes', None, '44000', 'France', 1370, 21000), (112, 'Signal Gift Stores', 'King', 'Jean', '7025551838', '8489 Strong St.', None, 'Las Vegas', 'NV', '83030', 'USA', 1166, 71800), (114, 'Australian Collectors, Co.', 'Ferguson', 'Peter', '03 9520 4555', '636 St Kilda Road', 'Level 3', 'Melbourne', 'Victoria', '3004', 'Australia', 1611, 117300)]


In [ ]:
llm = init_chat_model(
    model="openai/gpt-4o-mini",
    model_provider="openai",
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

In [12]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()
print(tools)

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001FA3880D790>), InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001FA3880D790>), ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001FA3880D790>), QuerySQLCheckerTool(description='Use this tool to dou

In [13]:
prompt_template = hub.pull('langchain-ai/sql-agent-system-prompt')
prompt_template.messages[0].pretty_print()

c:\Users\PC\Desktop\sql_agent\venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================ System Message ================================

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to th

In [ ]:
system_message = prompt_template.format(dialect='SQLite', top_k=5)

sql_agent = create_react_agent(llm, tools, prompt=system_message)

In [15]:
query = 'Get the top 10 selling products by quantity'

for event in sql_agent.stream(
    {"messages": ('user', query)},
    stream_mode='values'
):
    event['messages'][-1].pretty_print()

================================ Human Message =================================

Get the top 10 selling products by quantity
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_kWTRZ52zYCgi4LW6nAL7RskF)
 Call ID: call_kWTRZ52zYCgi4LW6nAL7RskF
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

customers, employees, offices, orderdetails, orders, payments, productlines, products
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_kWTRZ52zYCgi4LW6nAL7RskF)
 Call ID: call_kWTRZ52zYCgi4LW6nAL7RskF
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

customers, employees, offices, orderdetails, orders, payments, productlines, products
================================== Ai Message ==================================
Tool Calls:
  sql_d